# AVI Python SDK. CREATE, MODIFYING AND DELETING Objects

Having understood some basic concepts of how to use the sdk with the read operations, let's explore some operations that allow us to create, update and delete to have the complete picture.

 - https://avinetworks.com/docs/latest/api-guide/overview.html 
 - https://github.com/vmware/alb-sdk/tree/eng/python/avi/sdk 

Table of contents

- [1.- Login and Session Creation](#1--login-and-session-creation)
- [2.- Create (POST) Operations](#2--create-post-method)
- [3.- Modify (PUT) Operations](#3--modify-put-method)
- [4.- Delete (DELETE) Operations](#4--delete-delete-method)

# 1.- Login and Session Creation

As a first step, we need to create the ApiSession object to interact with the controller via API. 

In [6]:
from avi.sdk.avi_api import ApiSession
import datetime, time
from requests.packages import urllib3
urllib3.disable_warnings()
import json

# Import environment variables with controller information and credentials
from envs.controller_info import session_params as session_env

# Establish a first session with AVI Controller
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Update headers and api version imported from demo env file with controller version (ensure actual API Version is uses in subsequent requests
session_env['headers']['X-Avi-Version'] = api.remote_api_version['Version']
session_env['api_version'] = api.remote_api_version['Version']

# Create a new session with received AVI API Version
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Display Session ID to Verify AVI Controller Session Establishment
print('Successful connection to ' + session_env['name'] + '. Session ID:' + api.session_id)

Successful connection to avicontroller. Session ID:wbxkx26qg6ad3hpg788e1o3fnprllsv7


# 2.- CREATE (POST) Method

The POST method is used to create new objects. The way to invoke is quite similar to the GET method but this time we need to create a BODY document in JSON format ("doble quotes") containing the information we want to use as input for the object to be created. 

```json
body = {
      "name": "new_name",
      "cloud_ref": "http://192.168.1.15/cloud/cloud-19900-199910-1919"
}
```

<br><br>
Since error handling is important because we can make mistakes with fields names, it is crucial to add some logic to control AVI controller responses. In below example we will create a simple BODY manually including required fields to create a new vsvip object.

### Example 2.1:  Creating the BODY manually

Following example will create a manual body element to be pushed to the AVI Controller as part of the input of the POST request

In [3]:
vsvip_name = "vsvip-new"

body = {
  'vip': [
    {
      'enabled': 'true',
      'auto_allocate_ip': 'true',
      'auto_allocate_ip_type': 'V4_ONLY',
      'ipam_network_subnet': {
          'network_ref': 'https://192.168.1.15/api/network/network-5b30f803-879e-4312-b49b-309f41e98f7d',
          'subnet': {
            'ip_addr': {
               'addr': '192.168.1.0',
               'type': 'V4'
            },
            'mask': 24
        }
      },
      'vip_id': 1
    }
  ],
  'name': vsvip_name,
}

The POST method uses as ipunt arguments the path (API resource we want to interact with) and the BODY to be sent. To convert the body (python dictionary) into a JSON document that can be handled as part of the POST request, we need to convert it by using the **json.dumps** method. (json libraries must be imported in advanced)

In [8]:
# Define POST parameters
url_path="vsvip"

# Send POST information via POST
resp = api.post (url_path, data=json.dumps(body))

if resp.status_code in range(200, 299):
    print(resp)
    print('- New '+url_path+' named '+body['name'], resp.reason)#, resp.text)
else:
    print('Error in creating '+url_path+' :%s' % resp.text)

<Response [201]>
- New vsvip named vsvip-new CREATED


We can extract some information from the response received from the AVI Controller. In this example, we can show the allocated IP address from the internal IPAM. 

In [9]:
print("The allocated IP Address for the VSVIP object is: " + json.loads(resp.text)["vip"][0]["ip_address"]["addr"])

The allocated IP Address for the VSVIP object is: 192.168.1.51


### Example 2.2: Extracting references using api call instead of full values

A smarter way to populate some of the fields is by calling recurrently the API to get the value. As an example below, instead of providing the full network_ref value, we are just querying to the API where the name correspond to the intended network name _home-network_ with is much more simple way

In [10]:
vsvip_name = "vsvip-new-001"
network_name = "home-network"

body = {
  'vip': [
    {
      'enabled': 'true',
      'auto_allocate_ip': 'true',
      'auto_allocate_ip_type': 'V4_ONLY',
      'ipam_network_subnet': {
          'network_ref': '/api/network?name='+network_name,
          'subnet': {
            'ip_addr': {
               'addr': '192.168.1.0',
               'type': 'V4'
            },
            'mask': 24
        }
      },
      'vip_id': 1
    }
  ],
  'name': vsvip_name,
}

Now create the POST request using above data. The controller will resolve the value for the network_ref field using an internal query.  

In [11]:
# Define POST parameters
url_path="vsvip"

#Send BODY information via POST
resp = api.post (url_path, data=json.dumps(body))

if resp.status_code in range(200, 299):
    print(resp)
    print('- New '+url_path+' named '+body['name'], resp.reason)#, resp.text)
else:
    print('Error in creating '+url_path+' :%s' % resp.text)

#Shows allocated IP address.
print()
print("The allocated IP Address for the VSVIP object is: " + json.loads(resp.text)["vip"][0]["ip_address"]["addr"])

<Response [201]>
- New vsvip named vsvip-new-001 CREATED

The allocated IP Address for the VSVIP object is: 192.168.1.52


### Example 2.3.- Creating new object using a current object as a template

As you can tell, sometimes it could be really hard to craft the body from scratch since you need to be sure of all the mandatory fields and sintax. A better approach might be to extract (read) the information of an existing object and use this as a template. This allows you to clone JSON body and change the required information to create a new object without worrying that much about sintax and name of fields. 

Following example will use an existing serviceenginegroup object as template and after changing unique keys such as the name it will create a new cloned object. 

In [24]:
# Cloning an object from an exising one after changing required parameters
api_resource = "serviceenginegroup"
object_name = "SEG-TARGET-MAD-001"

# Get the existing Object to be used as template
template_data = api.get_object_by_name (api_resource, object_name) # Response to be used as template

Now we have the response we can create a new object with previous response. updating the required fields with new values as shown below:

In [34]:
# Override fields as per requirements
new_values = {
   "name": "NEW-SEGROUP-005",
   "max_vs_per_se": 22
}

# Create a new dictionary updating key-avlues as per new fields dictionary
template_data.update(new_values)

# Save updated dictionary into body variable
body = template_data

# Print modified items
print("Updated fields are: ")
for item in new_values: 
    print("- "+item+": " +str(body[item]))

Updated fields are: 
- name: NEW-SEGROUP-005
- max_vs_per_se: 22


Now we have the new body, we just need to invoke the POST method again pushing the new body as part of the request. 

In [35]:
#Send BODY information via POST
url_path = "serviceenginegroup"

resp = api.post (url_path, data=json.dumps(body))

if resp.status_code in range(200, 299):
    print(resp)
    print('- New '+url_path+' named '+body['name'], resp.reason)#, resp.text)
else:
    print('Error in creating '+url_path+' :%s' % resp.text)

<Response [201]>
- New serviceenginegroup named NEW-SEGROUP-005 CREATED


## 3.- Modify (PUT) Method

PUT is the method used to make modifications of an existing object. As in the POST method, a BODY containing the information we want to update is required.
<br><br>
Another important point is the uuid since the path consists on the url + the uuid of the target object.  

In this example we will change the name of an existing virtual service. We need to get the uuid before to proceed with name change. Also, override required fields (name in this case) with new values. 

In [45]:
# Get the Virtual Service configuration of a given name to extract uuid
object_name = "vs-example-org1"
object_new_name = "vs-example-org1-new"
object = api.get_object_by_name("virtualservice", object_name)

# Check if a non-empty response was received 
if object:
   # Extract uuid from response
   object_uuid = object["uuid"]
   # Override name field with new name
   object["name"] = object_new_name
   print("Data found for "+object_name+" with uuid "+object_uuid)

Data found for vs-example-org1 with uuid virtualservice-6bd64913-d5f4-41d7-ade9-eb2e7c003b23


In [41]:
# Define PUT parameters
url_path="virtualservice/"+vs_uuid
body = vs 

#Send BODY information via PUT
resp = api.put (url_path, data=json.dumps(body))

if resp.status_code in range(200, 299):
    print(resp)
    print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
    print()
else:
    print('Error in modifying '+url_path+' :%s' % resp.text)

<Response [200]>
- Object virtualservice/virtualservice-6bd64913-d5f4-41d7-ade9-eb2e7c003b23 named vs-example-org1 modified OK



## 4.- Delete (DELETE) Method

Last method is the delete method that, as you can guess is used for deleting an existing object. 

In [52]:
# Get the Service En configuration of a given name to extract uuid
api_resource = "serviceenginegroup"
object_name = "NEW-SEGROUP-005"
object = api.get_object_by_name("serviceenginegroup", object_name)

# Check if a non-empty response was received 
if object:
   # Extract uuid from response
   object_uuid = object["uuid"]
   # Override name field with new name
   print("Data found for "+object_name+" with uuid "+object_uuid)

Data found for NEW-SEGROUP-005 with uuid serviceenginegroup-7d6a3690-8aff-47d2-8d23-08577ed6bca2


In [53]:
# Define DELETE parameters
url_path="serviceenginegroup/"+object_uuid
body = object

#Send BODY information via DELETE
resp = api.delete(url_path)

if resp.status_code in range(200, 299):
    print(resp)
    print('- Object '+url_path+' named '+body['name']+ " deleted", resp.reason)#, resp.text)
    print()
else:
    print('Error in deleting '+url_path+' :%s' % resp.text)

<Response [204]>
- Object serviceenginegroup/serviceenginegroup-7d6a3690-8aff-47d2-8d23-08577ed6bca2 named NEW-SEGROUP-005 deleted NO CONTENT

